In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score
from tabulate import tabulate

In [2]:
df = pd.read_csv("rice data.csv")

In [3]:
df.head()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,ALLdaub4L,ALLdaub4a,ALLdaub4b,ALLdaub4Y,ALLdaub4Cb,ALLdaub4Cr,ALLdaub4XX,ALLdaub4YY,ALLdaub4ZZ,CLASS
0,7805,437.915,209.8215,48.0221,0.9735,99.6877,0.9775,7985,0.3547,4.3693,...,113.9924,65.0610,59.5989,104.8552,67.8779,63.0828,0.3673,0.3793,0.4733,Basmati
1,7503,340.757,138.3361,69.8417,0.8632,97.7400,0.9660,7767,0.6637,1.9807,...,105.7055,64.3685,62.2084,96.8375,65.5371,63.5832,0.3014,0.3144,0.3641,Arborio
2,5124,314.617,141.9803,46.5784,0.9447,80.7718,0.9721,5271,0.4760,3.0482,...,109.7155,62.6423,58.7439,100.2352,68.9753,59.8342,0.3233,0.3445,0.4448,Jasmine
3,7990,437.085,201.4386,51.2245,0.9671,100.8622,0.9659,8272,0.6274,3.9325,...,116.5405,64.9069,60.2562,107.2560,67.3298,63.2237,0.3880,0.4020,0.4904,Basmati
4,7433,342.893,140.3350,68.3927,0.8732,97.2830,0.9831,7561,0.6006,2.0519,...,107.7502,64.7071,61.3549,98.8704,66.2048,63.5378,0.3184,0.3303,0.3928,Arborio


In [4]:
df1 = df.iloc[:,0:16]
df1["CLASS"] = df["CLASS"]
df = df1

In [5]:
df.shape

(75000, 17)

In [6]:
df["CLASS"].value_counts()

CLASS
Basmati      15000
Arborio      15000
Jasmine      15000
Ipsala       15000
Karacadag    15000
Name: count, dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   AREA           75000 non-null  int64  
 1   PERIMETER      75000 non-null  float64
 2   MAJOR_AXIS     75000 non-null  float64
 3   MINOR_AXIS     75000 non-null  float64
 4   ECCENTRICITY   75000 non-null  float64
 5   EQDIASQ        75000 non-null  float64
 6   SOLIDITY       75000 non-null  float64
 7   CONVEX_AREA    75000 non-null  int64  
 8   EXTENT         75000 non-null  float64
 9   ASPECT_RATIO   75000 non-null  float64
 10  ROUNDNESS      75000 non-null  float64
 11  COMPACTNESS    75000 non-null  float64
 12  SHAPEFACTOR_1  75000 non-null  float64
 13  SHAPEFACTOR_2  75000 non-null  float64
 14  SHAPEFACTOR_3  75000 non-null  float64
 15  SHAPEFACTOR_4  75000 non-null  float64
 16  CLASS          75000 non-null  object 
dtypes: float64(14), int64(2), object(1)
memory usage: 

In [8]:
df.describe()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,ROUNDNESS,COMPACTNESS,SHAPEFACTOR_1,SHAPEFACTOR_2,SHAPEFACTOR_3,SHAPEFACTOR_4
count,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000
mean,8379.197507,378.169453,161.805540,66.829335,0.886077,101.731251,0.975896,8584.862320,0.633226,2.597063,0.732505,0.646079,0.020619,0.008407,0.429692,0.985509
std,3119.209274,70.597008,36.461005,16.689269,0.071906,17.874070,0.007966,3189.298025,0.123795,0.968982,0.138637,0.110787,0.005287,0.001903,0.141146,0.007280
min,3929.000000,261.040000,96.968300,34.673000,0.627700,70.728800,0.877500,4032.000000,0.278800,1.284500,0.392500,0.400600,0.011300,0.005100,0.160500,0.896200
25%,6259.000000,316.431500,132.623500,49.650200,0.846100,89.270400,0.970900,6385.000000,0.561000,1.876100,0.620600,0.551100,0.017000,0.006600,0.303700,0.981600
50%,7345.000000,351.261000,149.343950,69.183900,0.885600,96.705500,0.976400,7532.000000,0.655800,2.153200,0.775400,0.677100,0.018600,0.008700,0.458500,0.986400
75%,8901.000000,444.986000,197.462025,75.814125,0.950800,106.457100,0.982200,9153.000000,0.727800,3.228700,0.834500,0.725300,0.026200,0.009700,0.526100,0.990700
max,21019.000000,593.698000,255.647200,113.441100,0.986800,163.591600,0.992100,21633.000000,0.901700,6.179500,0.980000,0.879900,0.036900,0.013500,0.774300,0.999000


In [9]:
df = df.fillna(0)

In [10]:
print(sum(df.isnull().sum()))

0


In [11]:
df

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,ROUNDNESS,COMPACTNESS,SHAPEFACTOR_1,SHAPEFACTOR_2,SHAPEFACTOR_3,SHAPEFACTOR_4,CLASS
0,7805,437.915,209.8215,48.0221,0.9735,99.6877,0.9775,7985,0.3547,4.3693,0.5114,0.4751,0.0269,0.0062,0.2257,0.9863,Basmati
1,7503,340.757,138.3361,69.8417,0.8632,97.7400,0.9660,7767,0.6637,1.9807,0.8120,0.7065,0.0184,0.0093,0.4992,0.9888,Arborio
2,5124,314.617,141.9803,46.5784,0.9447,80.7718,0.9721,5271,0.4760,3.0482,0.6505,0.5689,0.0277,0.0091,0.3236,0.9865,Jasmine
3,7990,437.085,201.4386,51.2245,0.9671,100.8622,0.9659,8272,0.6274,3.9325,0.5256,0.5007,0.0252,0.0064,0.2507,0.9859,Basmati
4,7433,342.893,140.3350,68.3927,0.8732,97.2830,0.9831,7561,0.6006,2.0519,0.7944,0.6932,0.0189,0.0092,0.4806,0.9860,Arborio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74995,5551,285.911,114.1695,62.9079,0.8345,84.0699,0.9846,5638,0.6418,1.8149,0.8533,0.7364,0.0206,0.0113,0.5422,0.9841,Arborio
74996,7696,322.703,121.3900,81.1375,0.7438,98.9892,0.9868,7799,0.7309,1.4961,0.9287,0.8155,0.0158,0.0105,0.6650,0.9949,Karacadag
74997,7579,339.295,136.3125,71.2866,0.8524,98.2338,0.9805,7730,0.6399,1.9122,0.8273,0.7207,0.0180,0.0094,0.5193,0.9931,Arborio
74998,15174,489.502,200.9486,97.6282,0.8740,138.9969,0.9766,15537,0.7903,2.0583,0.7958,0.6917,0.0132,0.0064,0.4785,0.9848,Ipsala


In [12]:
df = np.array(df)
df = pd.DataFrame(df)
X = df.drop(columns = [16])
y = df[16]

sc = StandardScaler()
X = sc.fit_transform(X)
X = pd.DataFrame(X)

In [13]:
kf = KFold(n_splits=10, shuffle=False)

models = {
    "MLP": MLPClassifier(hidden_layer_sizes=(64, 32, 16, 8), activation='logistic', max_iter=10000, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(kernel='linear', random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=10),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42)
}

model_metrics = {name: {"Accuracy": [], "Error": [], "Precision": [], "Recall": [], "F1-Score": [], "Kappa": []} for name in models.keys()}

for fold, (train_idx, test_idx) in enumerate(kf.split(df)):
    print(f"Fold {fold + 1} :")
    
    X_train, X_test = X.iloc[train_idx, :], X.iloc[test_idx, :]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        error = 1 - acc  # Error rate
        prec = precision_score(y_test, y_pred, average="weighted", zero_division=0)
        rec = recall_score(y_test, y_pred, average="weighted")
        f1 = f1_score(y_test, y_pred, average="weighted")
        kappa = cohen_kappa_score(y_test, y_pred)

        model_metrics[name]["Accuracy"].append(acc)
        model_metrics[name]["Error"].append(error)
        model_metrics[name]["Precision"].append(prec)
        model_metrics[name]["Recall"].append(rec)
        model_metrics[name]["F1-Score"].append(f1)
        model_metrics[name]["Kappa"].append(kappa)

        print(f"{name}: Acc={acc:.4f}, Err={error:.4f}, Prec={prec:.4f}, Rec={rec:.4f}, F1={f1:.4f}, Kappa={kappa:.4f}")

    print("-" * 50)

table_data = [
    [name, 
     f"{np.mean(metrics['Accuracy']):.4f}", 
     f"{np.mean(metrics['Error']):.4f}", 
     f"{np.mean(metrics['Precision']):.4f}", 
     f"{np.mean(metrics['Recall']):.4f}", 
     f"{np.mean(metrics['F1-Score']):.4f}", 
     f"{np.mean(metrics['Kappa']):.4f}"]
    for name, metrics in model_metrics.items()
]

print(tabulate(table_data, headers=["Model", "Avg Acc", "Avg Error", "Avg Prec", "Avg Recall", "Avg F1", "Avg Kappa"], tablefmt="grid"))

Fold 1 :
MLP: Acc=0.9787, Err=0.0213, Prec=0.9787, Rec=0.9787, F1=0.9787, Kappa=0.9733
Random Forest: Acc=0.9787, Err=0.0213, Prec=0.9787, Rec=0.9787, F1=0.9787, Kappa=0.9733
SVM: Acc=0.9751, Err=0.0249, Prec=0.9751, Rec=0.9751, F1=0.9751, Kappa=0.9688
KNN: Acc=0.9784, Err=0.0216, Prec=0.9784, Rec=0.9784, F1=0.9784, Kappa=0.9730
Logistic Regression: Acc=0.9749, Err=0.0251, Prec=0.9750, Rec=0.9749, F1=0.9749, Kappa=0.9687
Decision Tree: Acc=0.9705, Err=0.0295, Prec=0.9705, Rec=0.9705, F1=0.9705, Kappa=0.9632
--------------------------------------------------
Fold 2 :
MLP: Acc=0.9836, Err=0.0164, Prec=0.9837, Rec=0.9836, F1=0.9836, Kappa=0.9795
Random Forest: Acc=0.9823, Err=0.0177, Prec=0.9823, Rec=0.9823, F1=0.9823, Kappa=0.9778
SVM: Acc=0.9803, Err=0.0197, Prec=0.9803, Rec=0.9803, F1=0.9803, Kappa=0.9753
KNN: Acc=0.9839, Err=0.0161, Prec=0.9839, Rec=0.9839, F1=0.9839, Kappa=0.9798
Logistic Regression: Acc=0.9800, Err=0.0200, Prec=0.9800, Rec=0.9800, F1=0.9800, Kappa=0.9750
Decision Tr

In [14]:
from sklearn.metrics import confusion_matrix

labels = np.unique(y)
conf_matrix_sum = {name: np.zeros((len(labels), len(labels))) for name in models.keys()}

for fold, (train_idx, test_idx) in enumerate(kf.split(df)):
    print(f"Fold {fold + 1} :")
    X_train, X_test = X.iloc[train_idx, :], X.iloc[test_idx, :]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        cm = confusion_matrix(y_test, y_pred, labels=labels)
        conf_matrix_sum[name] += cm
        print(f"{name}")

    print(f"Fold {fold + 1} Completed")
    print("-" * 50)

conf_matrix_total = {name: cm.astype(int) for name, cm in conf_matrix_sum.items()}

conf_matrix_data = []
for i, actual in enumerate(labels):
    for name, cm in conf_matrix_total.items():
        row = [actual, name]
        row.extend(cm[i])
        conf_matrix_data.append(row)

columns = ["Actual Class", "Algorithm"] + list(labels)
df_conf_matrix = pd.DataFrame(conf_matrix_data, columns=columns)

print(df_conf_matrix)

Fold 1 :
MLP
Random Forest
SVM
KNN
Logistic Regression
Decision Tree
Fold 1 Completed
--------------------------------------------------
Fold 2 :
MLP
Random Forest
SVM
KNN
Logistic Regression
Decision Tree
Fold 2 Completed
--------------------------------------------------
Fold 3 :
MLP
Random Forest
SVM
KNN
Logistic Regression
Decision Tree
Fold 3 Completed
--------------------------------------------------
Fold 4 :
MLP
Random Forest
SVM
KNN
Logistic Regression
Decision Tree
Fold 4 Completed
--------------------------------------------------
Fold 5 :
MLP
Random Forest
SVM
KNN
Logistic Regression
Decision Tree
Fold 5 Completed
--------------------------------------------------
Fold 6 :
MLP
Random Forest
SVM
KNN
Logistic Regression
Decision Tree
Fold 6 Completed
--------------------------------------------------
Fold 7 :
MLP
Random Forest
SVM
KNN
Logistic Regression
Decision Tree
Fold 7 Completed
--------------------------------------------------
Fold 8 :
MLP
Random Forest
SVM
KNN
Logist

In [15]:
print(tabulate(df_conf_matrix, headers='keys', tablefmt='grid'))

+----+----------------+---------------------+-----------+-----------+----------+-----------+-------------+
|    | Actual Class   | Algorithm           |   Arborio |   Basmati |   Ipsala |   Jasmine |   Karacadag |
+====+================+=====================+===========+===========+==========+===========+=============+
|  0 | Arborio        | MLP                 |     14528 |         0 |       13 |        35 |         424 |
+----+----------------+---------------------+-----------+-----------+----------+-----------+-------------+
|  1 | Arborio        | Random Forest       |     14502 |         0 |       12 |        28 |         458 |
+----+----------------+---------------------+-----------+-----------+----------+-----------+-------------+
|  2 | Arborio        | SVM                 |     14449 |         0 |        5 |        37 |         509 |
+----+----------------+---------------------+-----------+-----------+----------+-----------+-------------+
|  3 | Arborio        | KNN          